In [3]:
import numpy   as np
import pandas  as pd
import seaborn as sb

In [4]:
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext

from pyspark.sql.functions import *
from pyspark.sql.types     import *

In [5]:
spark = SparkSession\
        .builder\
        .appName('w261-final')\
        .master('local[*]')\
        .getOrCreate()

sc = spark.sparkContext
sq = SQLContext(sc)

In [6]:
!ls data

ls: cannot access data: No such file or directory


In [7]:
schema = StructType()

for f in range(0, 39 + 1):
    if   f == 0 : schema.add(StructField(f'CTR',        IntegerType(), True))
    elif f < 14 : schema.add(StructField(f'I{f   :02}', IntegerType(), True))
    else        : schema.add(StructField(f'C{f-13:02}',  StringType(), True))

In [9]:
train  = sq.read.format('csv')\
                .options(header = 'true', inferSchema = 'true', delimiter = '\t')\
                .schema(schema)\
                .load('../data/train.txt')

In [10]:
test   = sq.read.format('csv')\
                .options(header = 'true', inferSchema = 'true', delimiter = '\t')\
                .schema(schema[1:])\
                .load('../data/test.txt') 

In [11]:
num_features = [c for c in train.columns if 'I' in c]
cat_features = [c for c in train.columns if 'S' in c]

In [74]:
%%time
train_total = train.count()
test_total  = test.count()

CPU times: user 0 ns, sys: 3.9 ms, total: 3.9 ms
Wall time: 4.64 s


In [75]:
%%time
train[train.CTR == 1].count()/train_total

CPU times: user 3.34 ms, sys: 0 ns, total: 3.34 ms
Wall time: 6.23 s


0.25622338931920113

In [76]:
%%time
train.agg(*[(count(c)/train_total).alias(c) for c in train.columns]).toPandas().T

CPU times: user 105 ms, sys: 21.9 ms, total: 127 ms
Wall time: 54.1 s


,0
CTR,1.000000
I01,0.546394
I02,1.000000
I03,0.785355
I04,0.783219
I05,0.974191
I06,0.776348
I07,0.956744
I08,0.999503
I09,0.956744


In [78]:
%%time
train[[c for c in train.columns if 'I' in c or 'CTR' in c]].describe().toPandas().T

CPU times: user 0 ns, sys: 21.1 ms, total: 21.1 ms
Wall time: 16.6 s


,0,1,2,3,4
summary,count,mean,stddev,min,max
CTR,45840616,0.25622338931920113,0.4365466392520697,0,1
I01,25047060,3.5024134169838694,9.429076582074764,0,5775
I02,45840616,105.84842208490392,391.4578266505335,-3,257675
I03,36001169,26.913041629287093,397.9725885331938,0,65535
I04,35903247,7.322680452829238,8.79323075017961,0,969
I05,44657499,18538.99204906213,69394.60257569488,0,23159456
I06,35588288,116.06185400095671,382.566454284951,0,431037
I07,43857750,16.33313006253171,66.04975599781022,0,56311
I08,45817843,12.517042367097028,16.68888467758354,0,6047


Meeting Agenda Nov 8  docker-compose.yml   planning
README.md	      final_project.ipynb  w261Project2019.pdf
data		      notebooks


In [ ]:
train.agg(*(countDistinct(col(c)).alias(c) for c in train.columns)).show()

In [82]:
train.agg(countDistinct('C01')).show()

+-------------------+
|count(DISTINCT C01)|
+-------------------+
|               1460|
+-------------------+



In [12]:
num_data = train.select(num_features).toPandas()

axs = pd.scatter_matrix(num_data, figsize = (8, 8))
n = len(num_data.columns)
for i in range(n):
    v = axs[i, 0]
    v.yaxis.label.set_rotation(0)
    v.yaxis.label.set_ha('right')
    v.set_yticks(())
    h = axs[n-1, i]
    h.xaxis.label.set_rotation(90)
    h.set_xticks(())

Py4JJavaError: An error occurred while calling o120.collectToPython.
: org.apache.spark.SparkException: Job 0 cancelled because SparkContext was shut down
	at org.apache.spark.scheduler.DAGScheduler$$anonfun$cleanUpAfterSchedulerStop$1.apply(DAGScheduler.scala:932)
	at org.apache.spark.scheduler.DAGScheduler$$anonfun$cleanUpAfterSchedulerStop$1.apply(DAGScheduler.scala:930)
	at scala.collection.mutable.HashSet.foreach(HashSet.scala:78)
	at org.apache.spark.scheduler.DAGScheduler.cleanUpAfterSchedulerStop(DAGScheduler.scala:930)
	at org.apache.spark.scheduler.DAGSchedulerEventProcessLoop.onStop(DAGScheduler.scala:2128)
	at org.apache.spark.util.EventLoop.stop(EventLoop.scala:84)
	at org.apache.spark.scheduler.DAGScheduler.stop(DAGScheduler.scala:2041)
	at org.apache.spark.SparkContext$$anonfun$stop$6.apply$mcV$sp(SparkContext.scala:1949)
	at org.apache.spark.util.Utils$.tryLogNonFatalError(Utils.scala:1340)
	at org.apache.spark.SparkContext.stop(SparkContext.scala:1948)
	at org.apache.s